In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import os

In [6]:
song_features= pkl.load(open('vad_features_spotify.pkl', 'rb'))

'SONG' : ['Valence', 'Arousal']

In [7]:
song_features

{('Oscillate Wildly - 2011 Remaster', 'The Smiths'): [0.783, 0.791],
 ('Kokamiehen oikeus', 'Tukkanuotta'): [0.309, 0.973],
 ('Broken Heart', 'Boytronic'): [0.495, 0.521],
 ('New Rose', "Guns N' Roses"): [0.342, 0.965],
 ('Denounced', 'The Frames'): [0.707, 0.829],
 ('Crystal Wrists', 'Peter Murphy'): [0.633, 0.611],
 ('Carrion', 'Fiona Apple'): [0.179, 0.417],
 ('Return to the Warrens', 'Stuart Chatwood'): [0.563, 0.834],
 ('He About to Lose Me', 'Britney Spears'): [0.722, 0.797],
 ('Put You On (Feat. A Boogie Wit Da Hoodie)', 'PNB Rock'): [0.775, 0.902],
 ('Evacuation Code Deciphered', 'Arcturus'): [0.0818, 0.569],
 ('April Showers', 'Sugarland'): [0.121, 0.679],
 ('Tak Ska Du Ha', 'Handerre Linni'): [0.195, 0.502],
 ('Wealthy Humans', 'Céline Gillain'): [0.108, 0.744],
 ('Revolution (Beatles Cover)', 'Liferuiner'): [0.48, 0.959],
 ('One Flesh', 'Rome'): [0.168, 0.273],
 ('Still Sound', 'Toro y Moi'): [0.783, 0.551],
 ('#283', 'Hauschka'): [0.176, 0.188],
 ('Look Both Ways', 'Nonzus 

In [8]:
dir_1 = "./../dataset/1year_top500tracks_with_tags/"
file_list = os.listdir(dir_1)

In [19]:
def extract_playcount(user_file):
    user_file = open(user_file)
    temp = user_file.read()
    temp = temp.split('\n')
    tuple_set = set()
    for i in (temp[1:]):
        words = i.split(',')[:3]
        song_tuple = (words[0], words[1])
        if(song_tuple not in tuple_set):
            tuple_set.add(song_tuple)
    return tuple_set
    user_df = pd.read_csv(user_file)
    tuple_array = [(user_df['track'][i], user_df['artist'][i]) for i in range(len(user_df['track']))]
    play_count = dict()
    for i in tuple_array:
        if(i in play_count):
            play_count[i] += 1
        else:
            play_count[i] = 1
    return play_count

def extract_vad(play_count, song_features):
    tuples = list(play_count.keys())
    total_play = np.sum(list(play_count.values()))
    vad = np.array([0.0, 0.0])
    for i in tuples:
        if i in song_features:
            vad += song_features[i]*play_count[i]
        else:
            total_play -= play_count[i]
    vad /= total_play
    return vad


def non_weigthed_vad(user_file, song_features):
    file_1 = open(user_file)
    temp = file_1.read()
    temp = temp.split('\n')
    tuple_array = []
    for i in (temp[1:]):
        words = i.split(',')[:3]
        if(len(words) < 2):
            continue
        song_tuple = (words[0], words[1])
        tuple_array.append(song_tuple)
    temp_array = [[0, 0]]
    for i in tuple_array:
        if(i in song_features):
            temp_array.append(song_features[i])
    temp_array = temp_array[1:]
    return (np.mean(temp_array, axis=0))

# vad = extract_vad(extract_playcount('user.csv'), song_features)
# non_weigthed_vad('user.csv', song_features)
all_user_va_songs = dict()
for i in file_list:
    all_user_va_songs[i] = non_weigthed_vad(dir_1 + i, song_features)

In [20]:
all_user_va_songs

{'48e9a7f191695247670d622319fba921.csv': array([0.40087472, 0.64677849]),
 '1bc8474dc465d16f5c2a424b2a7427e7.csv': array([0.48954701, 0.79345136]),
 '2bdf82895c47ce2fd668fd6369da78db.csv': array([0.41477034, 0.52946535]),
 'cebe7bb75898cb69fe9bb992d10cf954.csv': array([0.48669383, 0.68128424]),
 '929e1073f2ec54bf04e18ca8f33a1724.csv': array([0.44303954, 0.60595444]),
 'e81cc3349b02197a403e412ad57157cc.csv': array([0.36866179, 0.55611245]),
 '3ae99483f8e0b14ff7c264625c96c507.csv': array([0.48262414, 0.63708753]),
 'd125a06735486fa936de1f856e01bbb4.csv': array([0.44504576, 0.85691741]),
 '003534c470fd624743ee3f7acef28a53.csv': array([0.44783991, 0.83974478]),
 '05ed17d0ecc78208ba5d6a3661a9fc95.csv': array([0.50977877, 0.71952558]),
 '20f3da7086adafce8a7adf3a85c67c7a.csv': array([0.42233682, 0.61051024]),
 'ae37da73621a312704f806a04664d5ae.csv': array([0.45652895, 0.54574053]),
 'ff7aefe14748d4cfed2a7f6cbc6cb226.csv': array([0.4372958 , 0.68057808]),
 '2fe1a2176412d3a3307a28047d16dfbc.csv

In [21]:
pkl.dump(all_user_va_songs, open('all_user_va_songs.pkl', 'wb'))